In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

# import numpy as np
# from sklearn.metrics import f1_score

# np.random.seed(0)

In [ ]:
# def correct(y_true,y_pred, threshold = 0.5):
#     y_pred = (y_pred > 0.5).astype('int32')
#     res = f1_score(y_true, y_pred, average = 'samples')
#     print(res)
#     return res

In [ ]:
# Creating Custom Metric
class MTCF1Score(tfa.metrics.FBetaScore):
    def __init__(self,num_classes, threshold = 0.5, beta = 1.0, name="mtc_f1_score", **kwargs):
        super().__init__(num_classes,threshold = threshold, beta = beta, name = name, **kwargs)
        self.mtc_f1_score = self.add_weight(name="mtc", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        super().update_state(y_true, y_pred, sample_weight=None)

    def result(self):
        f1_score = super().result()
        self.mtc_f1_score.assign(f1_score.shape[0] / tf.reduce_sum(1 / f1_score))
        return self.mtc_f1_score

    def reset_states(self):
        self.mtc_f1_score.assign(0.0)

In [ ]:
# Creating Custom Loss
class WeightedBinaryCrossEntropy(tf.keras.losses.Loss):
    def __init__(self, wp, wn, epsilon = 1e-7, name="weighted_binary_crossentropy"):
        super().__init__(name=name)
        self.wp = tf.cast(tf.constant(wp), tf.float32)
        self.wn = tf.cast(tf.constant(wn), tf.float32)
        self.epsilon = epsilon

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        loss = -tf.reduce_sum(tf.reduce_mean(tf.add(tf.multiply(self.wp ,tf.multiply(y_true, tf.math.log(y_pred + self.epsilon))), tf.multiply(self.wn,tf.multiply(1 - y_true, tf.math.log(1 - y_pred + self.epsilon)))),axis = 0))
        return loss

In [ ]:
# # Test 1 (ALL Predictions are zero)
# y_true = np.array([[0,0,0],[0,1,0],[1,1,0],[1,1,1],[0,0,1],[0,0,1],[0,0,1],[0,0,0],[0,0,1],[0,1,0]])
# y_pred = np.random.uniform(0.0,0.5,(10,3))
# correct(y_true, y_pred)

# m = MTCF1Score(3)
# m.update_state(y_true, y_pred)
# print(m.result().numpy())

# wbc = WeightedBinaryCrossEntropy([2,3,4],[1,2,3])
# print(wbc.call(y_true, y_pred))

# bc = tf.keras.losses.BinaryCrossentropy()
# print(bc(y_true, y_pred))

In [ ]:
# # Test 2 (ALL predictions 1)
# y_true = np.array([[0,0,0],[0,1,0],[1,1,0],[1,1,1],[0,0,1],[0,0,1],[0,0,1],[0,0,0],[0,0,1],[0,1,0]])
# y_pred = np.random.uniform(0.55,1.0,(10,3))
# correct(y_true, y_pred)

# m = MTCF1Score(3)
# m.update_state(y_true, y_pred)
# print(m.result().numpy())
# m.reset_states()
# print(m.result().numpy())

# wbc = WeightedBinaryCrossEntropy([2,3,4],[1,2,3])
# print(wbc.call(y_true, y_pred))

# bc = tf.keras.losses.BinaryCrossentropy()
# print(bc(y_true, y_pred))

In [ ]:
# # Test of loss Ans = -0.4956
# y_true = tf.constant(np.array(
#         [[1, 1, 1],
#          [1, 1, 0],
#          [0, 1, 0],
#          [1, 0, 1]]
#     ))
# # print("y_true:\n")
# # print(y_true)
    
# w_p = np.array([0.25, 0.25, 0.5])
# w_n = np.array([0.75, 0.75, 0.5])
# # print("\nw_p:\n")
# # print(w_p)

# # print("\nw_n:\n")
# # print(w_n)

# y_pred_1 = tf.constant(0.7 * np.ones(y_true.shape))
# # print("\ny_pred_1:\n")
# # print(y_pred_1)

# y_pred_2 = tf.constant(0.3 * np.ones(y_true.shape))
# # print("\ny_pred_2:\n")
# # print(y_pred_2)



# wbc = WeightedBinaryCrossEntropy(w_p,w_n)
# print(wbc.call(y_true, y_pred_2))
